Note: Kinfu in python does not work to a memory issues in opencv implementation (https://github.com/opencv/opencv_contrib/issues/2607)

In [1]:
import pyrealsense2 as rs
import numpy as np
import cv2
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import open3d as o3d
import matplotlib.pyplot as plt

In [2]:
cv2.__version__

'4.4.0'

In [3]:
focal_length = 643.338013 #942.8       # lense focal length, 1.88mm, 942.8 ???  643.338013, 643.096008
baseline = 55   #49.75  distance in mm between the two cameras
units = 0.512     # depth units, adjusted for the output to fit in one byte

width = 640
height = 480

intrinsicsNp = np.array(
    [[643.338013, 0, 638.95697 ],
     [  0, 643.096008, 402.330017],
     [  0, 0, 1.0 ]])

## FocalLengthLeft: 643.338379 643.096497
## PrincipalPointLeft: 638.957336 402.329956
## PinholeCameraIntrinsic(width: int, height: int, fx: float, fy: float, cx: float, cy: float)
pinhole_camera_intrinsic = o3d.camera.PinholeCameraIntrinsic(width, height, 
                                                             643.338013, 643.096008, 
                                                             638.956970, 402.330017);

print('Camera intrinsics:')
print(pinhole_camera_intrinsic.intrinsic_matrix)

Camera intrinsics:
[[643.338013   0.       638.95697 ]
 [  0.       643.096008 402.330017]
 [  0.         0.         1.      ]]


In [4]:
def getPointCloud(rgb_img, depth_image):
    # expects color image to be 8-bits per pixel
    # expects depth image to be 16-bits per pixel
    
    o3dColorImage1 = o3d.geometry.Image(rgb_img)
    o3dDepthImage1 = o3d.geometry.Image(depth_image)

    # Create an RGBD open3d Image
    frame_rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(o3dColorImage1, o3dDepthImage1, 
                                                                          convert_rgb_to_intensity=False)

    # Create Point cloud
    framePC = o3d.geometry.PointCloud.create_from_rgbd_image(frame_rgbd_image, pinhole_camera_intrinsic)
    
    
#   framePC = o3d.geometry.PointCloud.create_from_depth_image(o3dDepthImage1, pinhole_camera_intrinsic, 
#                                                                   depth_scale=1000.0)
    
    return framePC

In [5]:
cv2.setUseOptimized(False)

In [6]:
params = cv2.kinfu.Params_defaultParams();
params.frameSize = (width, height)
params.depthFactor = 5000.0
params.intr = intrinsicsNp

print(params.frameSize)
print(params.intr)
print(params.depthFactor)
print(params.bilateral_kernel_size)
print(params.bilateral_sigma_depth)
print(params.bilateral_sigma_spatial)
print(params.pyramidLevels)
print(params.voxelSize)

kinfu = cv2.kinfu.KinFu_create(params)

(640, 480)
[[643.338     0.      638.957  ]
 [  0.      643.096   402.33002]
 [  0.        0.        1.     ]]
5000.0
7
0.03999999910593033
4.5
3
0.005859375


In [7]:
# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()

res_x = width
res_y = height

config.enable_stream(rs.stream.depth, res_x, res_y, rs.format.z16, 30)
config.enable_stream(rs.stream.color, res_x, res_y, rs.format.bgr8, 30)

config.enable_stream(rs.stream.infrared, 1, res_x, res_y, rs.format.y8, 30)
config.enable_stream(rs.stream.infrared, 2, res_x, res_y, rs.format.y8, 30)

# Start streaming
pipeline_profile = pipeline.start(config)

## To set laser on/off or set laser power
device = pipeline_profile.get_device()
depth_sensor = device.query_sensors()[0]
laser_pwr = depth_sensor.get_option(rs.option.laser_power)
print("laser power = ", laser_pwr)
laser_range = depth_sensor.get_option_range(rs.option.laser_power)
print("laser power range = " , laser_range.min , "~", laser_range.max)
depth_sensor.set_option(rs.option.laser_power, 150)


spatial = rs.spatial_filter()
spatial.set_option(rs.option.filter_magnitude, 3)
spatial.set_option(rs.option.filter_smooth_alpha, 0.5)
spatial.set_option(rs.option.filter_smooth_delta, 20)
spatial.set_option(rs.option.holes_fill, 0)
temporal = rs.temporal_filter()
# decimation = rs.decimation_filter()
# decimation.set_option(rs.option.filter_magnitude, 4)
hole_filling = rs.hole_filling_filter()
hole_filling.set_option(rs.option.holes_fill, 1)

# Image directory 
directory = r'./outputs/'
img_count = 1

# Reset Kinfu
kinfu.reset()

cc = 0
try:
    while True:
        # Wait for a coherent pair of frames: depth and color
        frames = pipeline.wait_for_frames()
        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()
        ir1_frame = frames.get_infrared_frame(1) # Left IR Camera, it allows 0, 1 or no input
        ir2_frame = frames.get_infrared_frame(2) # Right IR camera
        if not depth_frame or not color_frame:
            continue

        if not ir1_frame or not ir2_frame:
            continue
            
        # skipping first 20 frames, to wait for sensor for warm up
        cc = cc + 1
        if cc < 20:
            continue

        # Convert images to numpy arrays
        depth_image = np.asanyarray(depth_frame.get_data()) # dtype of print(depth_image.dtype) is: uint16
        color_image = np.asanyarray(color_frame.get_data()) # FORMAT: BGR, dtype of print(color_image.dtype) is: uint8
        color_image_rgb = cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB)
        
        ir1_image = np.asanyarray(ir1_frame.get_data()) # Left image
        ir2_image = np.asanyarray(ir2_frame.get_data()) # Right image
        
        left = ir1_image
        right = ir2_image
        rs_depth = depth_image # (image of 16-bit per pixel)
        
        ## Apply filters to real-sense depth
        filtered_rs_depth = spatial.process(depth_frame)
        filtered_rs_depth = temporal.process(filtered_rs_depth)
        filtered_rs_depth = hole_filling.process(filtered_rs_depth)
        filtered_rs_depth = np.asanyarray(filtered_rs_depth.get_data()) # dtype of print(filtered_rs_depth.dtype) is: uint16
        
        ## Apply bilateral filter to RS depth image BEFORE RS filters
        tmp = rs_depth.astype(np.float32)
        tmp = cv2.bilateralFilter(tmp, 9, 75, 75)
        bilateral_depth = tmp.astype(np.uint16)
        
        ## Apply bilateral filter to RS depth image AFTER RS filters
        tmp = filtered_rs_depth.astype(np.float32)
        tmp = cv2.bilateralFilter(tmp, 9, 75, 75)
        filtered_bilateral_depth = tmp.astype(np.uint16)
        
        
#         print('# of valid depth values [rs_depth]: ', len(rs_depth[rs_depth > 0]))
#         print(rs_depth[rs_depth > 0])
#         print('# of valid depth values [bilateral_depth]: ', len(bilateral_depth[bilateral_depth > 0]))
#         print(bilateral_depth[bilateral_depth > 0])
        
#         print('# of valid depth values [filtered_rs_depth]: ', len(filtered_rs_depth[filtered_rs_depth > 0]))
#         print(filtered_rs_depth[filtered_rs_depth > 0])
#         print('# of valid depth values [filtered_bilateral_depth]: ', len(filtered_bilateral_depth[filtered_bilateral_depth > 0]))
#         print(filtered_bilateral_depth[filtered_bilateral_depth > 0])
        
    
        result = kinfu.update(filtered_rs_depth)
        print(result)
#         out = np.zeros(shape=filtered_rs_depth.shape).astype(np.float32)
        pose = cv2.kinfu_KinFu.getPose()
#         print(pose)


                    
            
            
            

        #### VISUALISATION ####
        is_visual_on = False
        if not is_visual_on:
            continue
        
        ## Visualize RGB frame
#         cv2.namedWindow('RealSense', cv2.WINDOW_AUTOSIZE)
#         cv2.imshow('RealSense', color_image)
        
        ## Visualize IR frames
        ir_images = np.hstack((ir1_image, ir2_image))
        cv2.namedWindow('IRSense', cv2.WINDOW_AUTOSIZE)
        cv2.imshow('IRSense', ir_images)
        
        
        
        
        ## Visualize Real sense depth frames
        ## Apply colormap on Real-sense depth image 
        rs_depth_sccaled  = cv2.convertScaleAbs(depth_image, alpha=0.03)  # (image converted to 8-bit per pixel)
        filtered_rs_depth_scaled = cv2.convertScaleAbs(filtered_rs_depth, alpha=0.03)  # (image converted to 8-bit per pixel)
        #depth_colormap = cv2.applyColorMap(rs_depth_sccaled, cv2.COLORMAP_JET)
        depth_colormap = cv2.applyColorMap(cv2.equalizeHist(rs_depth_sccaled), cv2.COLORMAP_JET)
        #filtered_depth_colormap = cv2.applyColorMap(filtered_rs_depth_scaled, cv2.COLORMAP_JET)
        filtered_depth_colormap = cv2.applyColorMap(cv2.equalizeHist(filtered_rs_depth_scaled), cv2.COLORMAP_JET)
        both_depths2 = np.hstack((depth_colormap, filtered_depth_colormap))
        cv2.namedWindow('RS_DepthMap', cv2.WINDOW_AUTOSIZE)
        cv2.imshow('RS_DepthMap', both_depths2)
        
        
        
        
        ## Visualize Real sense depth frames with Bilateral filter
        bilateral_depth_colormap = cv2.applyColorMap(cv2.equalizeHist(cv2.convertScaleAbs(bilateral_depth, alpha=0.03)), cv2.COLORMAP_JET)
        bilateral_filtered_depth_colormap = cv2.applyColorMap(cv2.equalizeHist(cv2.convertScaleAbs(filtered_bilateral_depth, alpha=0.03)), cv2.COLORMAP_JET)
        both_depths2 = np.hstack((bilateral_depth_colormap, bilateral_filtered_depth_colormap))
        cv2.namedWindow('RS_Bilateral_DepthMap', cv2.WINDOW_AUTOSIZE)
        cv2.imshow('RS_Bilateral_DepthMap', both_depths2)
        
        
        
        
#         ## Visualize RealSense refined disparity and depth maps
        tmp1 = cv2.convertScaleAbs(refined_rs_disparity.astype(np.uint16))
        o3 = cv2.applyColorMap(cv2.equalizeHist(tmp1), cv2.COLORMAP_JET)
        o3[refined_rs_disparity < 0] = 0
        o4 = cv2.applyColorMap(cv2.equalizeHist(cv2.convertScaleAbs(refined_rs_depth, alpha=0.03)), cv2.COLORMAP_JET)
        o4[refined_rs_depth < 0] = 0
        stn_images1 = np.hstack((o3, o4))
        cv2.namedWindow('Refined', cv2.WINDOW_AUTOSIZE)
        cv2.imshow('Refined', stn_images1)
        
        
        
        
        
        
        ## Visualize stereoNet disparity and depth maps (StereoNET)
        tmp1 = cv2.convertScaleAbs(sn_disparity.astype(np.uint16))
        o1 = cv2.applyColorMap(cv2.equalizeHist(tmp1), cv2.COLORMAP_JET)
        o1[sn_disparity < 0] = 0
        o2 = cv2.applyColorMap(cv2.equalizeHist(cv2.convertScaleAbs(sn_depth_map, alpha=0.03)), cv2.COLORMAP_JET)
        o2[sn_depth_map < 0] = 0
        #stn_images = o1
        stn_images = np.hstack((o1, o2))
        cv2.namedWindow('StereoNet', cv2.WINDOW_AUTOSIZE)
        cv2.imshow('StereoNet', stn_images)
        
        
        
        
        

        key = cv2.waitKey(1)
        # Press esc or 'q' to close the image window
        if key & 0xFF == ord('q') or key == 27:
            cv2.destroyAllWindows()
            break
        if key == 116:
#             cv2.imwrite(directory + str(img_count) + '_color_image.jpg', color_image)
#             cv2.imwrite(directory + str(img_count) + '_left.jpg', left)
#             cv2.imwrite(directory + str(img_count) + '_right.jpg', right)
#             cv2.imwrite(directory + str(img_count) + '_rs_depth.jpg', depth_colormap)
#             cv2.imwrite(directory + str(img_count) + '_rs_filtered_depth.jpg', filtered_depth_colormap)
#             cv2.imwrite(directory + str(img_count) + '_bm_depth.jpg', temp1)
#             cv2.imwrite(directory + str(img_count) + '_bm_filtered_depth.jpg', temp2)
#             cv2.imwrite(directory + str(img_count) + '_bm_disparity.jpg', raw_disparity_scaled)
#             cv2.imwrite(directory + str(img_count) + '_bm_filtered_disparity.jpg', filtered_disparity_scaled)
            img_count = img_count+1
        if key == ord('1'):
            rs_pc = getPointCloud(color_image_rgb, rs_depth)
            rs_pc.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
            print('rs_pc', rs_pc)
            o3d.visualization.draw_geometries([rs_pc])
        if key == ord('2'):
            rs_pc = getPointCloud(color_image_rgb, filtered_rs_depth)
            rs_pc.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
            print('rs_pc', rs_pc)
            o3d.visualization.draw_geometries([rs_pc])
        if key == ord('3'):
            rs_pc = getPointCloud(color_image_rgb, bilateral_depth)
            rs_pc.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
            print('rs_pc', rs_pc)
            o3d.visualization.draw_geometries([rs_pc])
        if key == ord('4'):
            rs_pc = getPointCloud(color_image_rgb, sn_depth_map)
            rs_pc.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
            print('rs_pc', rs_pc)
            o3d.visualization.draw_geometries([rs_pc])
        if key == ord('5'):
            rs_pc = getPointCloud(color_image_rgb, refined_rs_depth)
            rs_pc.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
            print('rs_pc', rs_pc)
            o3d.visualization.draw_geometries([rs_pc])
            
finally:

    # Stop streaming
    pipeline.stop()

laser power =  150.0
laser power range =  0.0 ~ 360.0
True


AttributeError: type object 'cv2.kinfu_KinFu' has no attribute 'getPose'